In [136]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import  f_regression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
import seaborn as sns



In [217]:
dataset=pd.read_csv("bengaluru_house_prices.csv")

In [218]:
dataset

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...,...
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,ArsiaEx,3453,4.0,0.0,231.00
13316,Super built-up Area,Ready To Move,Richards Town,4 BHK,NaN,3600,5.0,NaN,400.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,Mahla T,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,SollyCl,4689,4.0,1.0,488.00


In [223]:
dataset.drop(['availability','society'],axis=1,inplace=True)

KeyError: "['availability', 'society'] not found in axis"

In [224]:
dataset

,area_type,location,size,total_sqft,bath,balcony,price
0,Super built-up Area,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07
1,Plot Area,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00
2,Built-up Area,Uttarahalli,3 BHK,1440,2.0,3.0,62.00
3,Super built-up Area,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00
4,Super built-up Area,Kothanur,2 BHK,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...
13315,Built-up Area,Whitefield,5 Bedroom,3453,4.0,0.0,231.00
13316,Super built-up Area,Richards Town,4 BHK,3600,5.0,NaN,400.00
13317,Built-up Area,Raja Rajeshwari Nagar,2 BHK,1141,2.0,1.0,60.00
13318,Super built-up Area,Padmanabhanagar,4 BHK,4689,4.0,1.0,488.00


In [225]:
print(dataset.columns)

Index(['area_type', 'location', 'size', 'total_sqft', 'bath', 'balcony',
       'price'],
      dtype='object')


In [226]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
if 'location' in dataset.columns:
    dataset['location'] = encoder.fit_transform(dataset['location'])
else:
    print("The 'location' column does not exist in the dataset.")


In [227]:
dataset["area_type"].value_counts()

area_type
Super built-up  Area    8790
Built-up  Area          2418
Plot  Area              2025
Carpet  Area              87
Name: count, dtype: int64

In [228]:
categorical_columns = dataset.select_dtypes(include=['object', 'category']).columns
print(categorical_columns)


Index(['area_type', 'size', 'total_sqft'], dtype='object')


In [229]:
dataset["balcony"].value_counts()

balcony
2.0    5113
1.0    4897
3.0    1672
0.0    1029
Name: count, dtype: int64

In [230]:
dataset["size"].value_counts()

size
2 BHK         5199
3 BHK         4310
4 Bedroom      826
4 BHK          591
3 Bedroom      547
1 BHK          538
2 Bedroom      329
5 Bedroom      297
6 Bedroom      191
1 Bedroom      105
8 Bedroom       84
7 Bedroom       83
5 BHK           59
9 Bedroom       46
6 BHK           30
7 BHK           17
1 RK            13
10 Bedroom      12
9 BHK            8
8 BHK            5
11 BHK           2
11 Bedroom       2
10 BHK           2
14 BHK           1
13 BHK           1
12 Bedroom       1
27 BHK           1
43 Bedroom       1
16 BHK           1
19 BHK           1
18 Bedroom       1
Name: count, dtype: int64

In [231]:
#converting object into float in total_sqft column
def process_sqft(value):
    if '-' in value:
        vals = value.split('-')
        return (float(vals[0]) + float(vals[1])) / 2  # Calculate the average
    try:
        return float(value)  # Try converting directly if it's a single number
    except ValueError:
        return None  # Return None for invalid entries

dataset['total_sqft'] = dataset['total_sqft'].apply(process_sqft)


In [232]:
dataset["total_sqft"].nunique()

1999

In [233]:
dataset["total_sqft"]=dataset["total_sqft"].astype('float64')

In [234]:
dataset["total_sqft"].dtype

dtype('float64')

In [235]:
dataset["total_sqft"].fillna(dataset["total_sqft"].mean(),inplace=True)


C:\Users\argka\AppData\Local\Temp\ipykernel_2168\2456738064.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["total_sqft"].fillna(dataset["total_sqft"].mean(),inplace=True)


In [236]:
dataset.dtypes

area_type      object
location        int32
size           object
total_sqft    float64
bath          float64
balcony       float64
price         float64
dtype: object

In [237]:
dataset.isnull().sum()

area_type       0
location        0
size           16
total_sqft      0
bath           73
balcony       609
price           0
dtype: int64

In [238]:
dataset["size"].fillna(dataset["size"].mode()[0],inplace=True)

C:\Users\argka\AppData\Local\Temp\ipykernel_2168\2790918884.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["size"].fillna(dataset["size"].mode()[0],inplace=True)


In [239]:
dataset["location"].fillna("NA",inplace=True)


C:\Users\argka\AppData\Local\Temp\ipykernel_2168\440914251.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["location"].fillna("NA",inplace=True)


In [240]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   area_type   13320 non-null  object 
 1   location    13320 non-null  int32  
 2   size        13320 non-null  object 
 3   total_sqft  13320 non-null  float64
 4   bath        13247 non-null  float64
 5   balcony     12711 non-null  float64
 6   price       13320 non-null  float64
dtypes: float64(4), int32(1), object(2)
memory usage: 676.5+ KB


In [241]:
dataset["bath"].fillna(dataset["bath"].mean(),inplace=True)
dataset["balcony"].fillna(dataset["balcony"].mean(),inplace=True)

C:\Users\argka\AppData\Local\Temp\ipykernel_2168\2523768286.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["bath"].fillna(dataset["bath"].mean(),inplace=True)
C:\Users\argka\AppData\Local\Temp\ipykernel_2168\2523768286.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

In [242]:
def convert_to_numeric(value):
    try:
        return float(value)
    except ValueError:
        # Handle ranges like '2100 - 2850' by averaging
        if '-' in value:
            parts = value.split('-')
            return (float(parts[0]) + float(parts[1])) / 2
        return None  # Return None for invalid values

dataset['total_sqft'] = dataset['total_sqft'].apply(convert_to_numeric)


In [243]:
#converting categorical into numerical datatype
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
encoder = LabelEncoder()

# Convert categorical column to numerical
dataset["area_type"] = encoder.fit_transform(dataset["area_type"])
dataset["size"] = encoder.fit_transform(dataset["size"])


In [244]:
dataset.shape

(13320, 7)

In [245]:
dataset.describe()

,area_type,location,size,total_sqft,bath,balcony,price
count,13320.000000,13320.000000,13320.000000,13320.000000,13320.000000,13320.000000,13320.000000
mean,0.977027,672.659009,14.791291,1559.626694,2.692610,1.584376,112.565627
std,0.582742,365.024806,4.455820,1236.264860,1.337777,0.798360,148.971674
min,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,8.000000
25%,1.000000,397.000000,13.000000,1100.000000,2.000000,1.000000,50.000000
50%,1.000000,654.000000,16.000000,1279.500000,2.000000,2.000000,72.000000
75%,1.000000,999.000000,16.000000,1678.000000,3.000000,2.000000,120.000000
max,2.000000,1305.000000,30.000000,52272.000000,40.000000,3.000000,3600.000000


In [254]:
dataset

,area_type,location,size,total_sqft,bath,balcony,price
0,1,419,13,1056.0,2.0,1.000000,39.07
1,2,317,19,2600.0,5.0,3.000000,120.00
2,0,1179,16,1440.0,2.0,3.000000,62.00
3,1,757,16,1521.0,3.0,1.000000,95.00
4,1,716,13,1200.0,2.0,1.000000,51.00
...,...,...,...,...,...,...,...
13315,0,1252,22,3453.0,4.0,0.000000,231.00
13316,1,1004,18,3600.0,5.0,1.584376,400.00
13317,0,972,13,1141.0,2.0,1.000000,60.00
13318,1,907,18,4689.0,4.0,1.000000,488.00


In [258]:
dataset1=dataset


In [259]:
dataset1

,area_type,location,size,total_sqft,bath,balcony,price
0,1,419,13,1056.0,2.0,1.000000,39.07
1,2,317,19,2600.0,5.0,3.000000,120.00
2,0,1179,16,1440.0,2.0,3.000000,62.00
3,1,757,16,1521.0,3.0,1.000000,95.00
4,1,716,13,1200.0,2.0,1.000000,51.00
...,...,...,...,...,...,...,...
13315,0,1252,22,3453.0,4.0,0.000000,231.00
13316,1,1004,18,3600.0,5.0,1.584376,400.00
13317,0,972,13,1141.0,2.0,1.000000,60.00
13318,1,907,18,4689.0,4.0,1.000000,488.00


In [262]:
df=pd.DataFrame(dataset1)


In [263]:
df

,area_type,location,size,total_sqft,bath,balcony,price
0,1,419,13,1056.0,2.0,1.000000,39.07
1,2,317,19,2600.0,5.0,3.000000,120.00
2,0,1179,16,1440.0,2.0,3.000000,62.00
3,1,757,16,1521.0,3.0,1.000000,95.00
4,1,716,13,1200.0,2.0,1.000000,51.00
...,...,...,...,...,...,...,...
13315,0,1252,22,3453.0,4.0,0.000000,231.00
13316,1,1004,18,3600.0,5.0,1.584376,400.00
13317,0,972,13,1141.0,2.0,1.000000,60.00
13318,1,907,18,4689.0,4.0,1.000000,488.00


In [264]:
# Save DataFrame to CSV
df.to_csv('preprocessed_data.csv', index=False)

print("Preprocessed DataFrame saved as preprocessed_data.csv")


Preprocessed DataFrame saved as preprocessed_data.csv
